# Dataset Preparations before running Evaluation pipeline

## minimal_example

* Simply resize all images

In [ ]:
import os
from PIL import Image

for path, dirnames, filenames in os.walk("./data/minimal_example"):
    for fn in filenames:
        im = Image.open(f"{path}/{fn}")
        im = im.resize((224, 224))
        im.save(f"{path}/{fn}")

## SynthNet - Rendered Datasets from Topex CAD files

1. Prepend 'synth_' to image names and fix appended index with zeros
2. Move images into class label directory

In [ ]:
import os
import shutil

IN_DIR = '/mnt/c/Users/denni/Desktop/work/7054-12-300-l_drucker_se_su_st_st_512_32/render'
# IN_DIR = '/home/dennis/Desktop/work/7054-12-300-l_drucker_se_su_st_st_512_32/render'
OUT_DIR = 'data/synthnet/train/7054-12-300-l_drucker_se_su_st_st_512_32'

def prepare_synthnet_synth_imgs(in_dir: str, out_dir: str) -> None:
    os.makedirs(f"{out_dir}/images", exist_ok=True)
    os.makedirs(f"{out_dir}/assets", exist_ok=True)
    synth_ims = os.listdir(f'{in_dir}')
    for i, im_name in enumerate(synth_ims):
        im_path = f'{in_dir}/{im_name}'
        im_class = '_'.join(im_name.split('_')[:-1])
        im_oid = 1
        im_label_i = int(im_name.split('_')[-1].split('.')[0])

        im_name_new = f'train_{im_class}_{im_oid:04d}_{im_label_i:04d}.png'
        dest_path = f'{out_dir}/images/{im_class}/{im_class}_{im_oid:04d}/{im_name_new}'.lower()

        os.makedirs(f'{out_dir}/images/{im_class}/{im_class}_{im_oid:04d}'.lower(), exist_ok=True)
        shutil.copy(im_path, dest_path)

prepare_synthnet_synth_imgs(IN_DIR, OUT_DIR)

## SynthNet - bounding box annotated topex-real-N datasets

1.  Parse images from annotations.json file
2.  Get objects' bounding box dimensions
3.  Crop image with bounding box dimensions and expand the smaller side to get a rectangular crop 
    or expand to image dimensions if the image is too small. 
4.  Resize image to target_size


In [ ]:
import os
import shutil
import json
from typing import Tuple
from PIL import Image, ImageOps

IN_DIR = '/home/dennis/Desktop/synthnet/topex-real-123'
OUT_DIR = 'data/synthnet/real/topex-real-123_pb_256/images'
TARGET_SIZE = (256, 256) 
CROP_OPTION = 'padding_black' # one of ['resize', 'padding_black']

# NOTE: Script does not handle duplicate part_ids at the moment!
def prepare_synthnet_topex_real_imgs(in_dir: str, out_dir: str, target_size: Tuple, crop_option: str) -> None:
    os.makedirs(out_dir, exist_ok=True)
    with open(f'{in_dir}/annotations.json', 'r', encoding='utf-8') as f:
        annotations = json.load(f)

    for annotation in annotations:
        for annotation_label in annotation['label']:
            im_label = annotation_label['rectanglelabels'][0]

            im_name = annotation['image'].split('/')[-1]
            im_path = f'{in_dir}/{im_label}/{im_name}'
            if not os.path.exists(im_path):
                continue
            im = Image.open(im_path)
            # Rotate image as expected
            im = ImageOps.exif_transpose(im)

            # Crop
            im_w, im_h = annotation_label['original_width'], annotation_label['original_height']
            # Bounding box position and size is given in percent (0-100)
            bb_x = int((annotation_label['x'] / 100) * im_w)
            bb_y = int((annotation_label['y'] / 100) * im_h)
            bb_w = int((annotation_label['width'] / 100) * im_w)
            bb_h = int((annotation_label['height'] / 100) * im_h)

            # get rectangular bounding box image dimension in px, expand narrow axis to wider axis size
            if bb_w >= bb_h:
                d = (bb_w - bb_h) / 2
                left = bb_x
                top = bb_y - d
                right = bb_x + bb_w
                bottom = bb_y + bb_h + d
            if bb_w < bb_h:
                d = (bb_h - bb_w) / 2
                left = bb_x - d
                top = bb_y
                right = bb_x + bb_w + d
                bottom = bb_y + bb_h

            # Limit crop to image dimensions if !padding
            if crop_option == 'resize':
                w, h = im.size
                left = 0 if left < 0 else left
                top = 0 if top < 0 else top
                right = w if right > w else right
                bottom = h if bottom > h else bottom

            im = im.crop((int(left), int(top), int(right), int(bottom)))
            im = im.resize(target_size)

            os.makedirs(f'{out_dir}/{im_label}'.lower(), exist_ok=True)
            im.save(f'{out_dir}/{im_label}/test_{im_name}'.lower())

prepare_synthnet_topex_real_imgs(IN_DIR, OUT_DIR, TARGET_SIZE, CROP_OPTION)
# for path, dns, fns in os.walk("./data/synthnet/test/topex-real-123_pb_256/images"):
#     for fn in fns:
#         new_fn = f'test{fn[4:]}'
#         shutil.move(f'{path}/{fn}', f"{path}/{new_fn}")

## ShapeNetCore17 <-> ILSVRC

We use renders of **ShapeNetCore objects** from 17 classes as train set and to build the search index and connect them to **real fotos from the ImageNet Dataset** using their **synset connections**

```
02747177 ['ashcan', 'trash can', 'garbage can', 'wastebin', 'ash bin', 'ash-bin', 'ashbin', 'dustbin', 'trash barrel', 'trash bin']
02808440 ['bathtub', 'bathing tub', 'bath', 'tub']
02843684 ['birdhouse']
02992529 ['cellular telephone', 'cellular phone', 'cellphone', 'cell', 'mobile phone']
03085013 ['computer keyboard', 'keypad']
03207941 ['dishwasher', 'dish washer', 'dishwashing machine']
03337140 ['file', 'file cabinet', 'filing cabinet']
03642806 ['laptop', 'laptop computer']
03691459 ['loudspeaker', 'speaker', 'speaker unit', 'loudspeaker system', 'speaker system']
03710193 ['mailbox', 'letter box']
03759954 ['microphone', 'mike']
03761084 ['microwave', 'microwave oven']
03938244 ['pillow']
03991062 ['pot', 'flowerpot']
04074963 ['remote control', 'remote']
04330267 ['stove']
04554684 ['washer', 'automatic washer', 'washing machine']
```

In [ ]:
import os
import shutil

IN_DIR = '/home/dennis/Desktop/sn17_renders'
OUT_DIR = 'data/sn17ilsvrc/train'

# TODO update 
# def prepare_synthnet_synth_imgs(in_dir: str, out_dir: str) -> None:
#     os.makedirs(f"{out_dir}/images", exist_ok=True)
#     os.makedirs(f"{out_dir}/assets", exist_ok=True)
#     synth_ims = os.listdir(f'{in_dir}')
#     for i, im_name in enumerate(synth_ims):
#         im_path = f'{in_dir}/{im_name}'
#         im_class = '_'.join(im_name.split('_')[:-1])
#         im_oid = 1
#         im_label_i = int(im_name.split('_')[-1].split('.')[0])

#         im_name_new = f'train_{im_class}_{im_oid:04d}_{im_label_i:04d}.png'
#         dest_path = f'{out_dir}/images/{im_class}/{im_oid:04d}/{im_name_new}'.lower()

#         os.makedirs(f'{out_dir}/images/{im_class}/{im_oid:04d}'.lower(), exist_ok=True)
#         shutil.copy(im_path, dest_path)

# prepare_synthnet_synth_imgs(IN_DIR, OUT_DIR)

def prepare_synthnet_synth_imgs(in_dir: str, out_dir: str) -> None:
    os.makedirs(out_dir, exist_ok=True)
    synth_ims = os.listdir(f'{in_dir}/render')
    for i, im_name in enumerate(synth_ims):
        im_path = f'{in_dir}/render/{im_name}'
        
        im_entity = '_'.join(im_name.split('_')[:-1])
        im_entity_i = int(im_name.split('_')[-1].split('.')[0])

        for synset in os.listdir('../notebooks/data/synthnet/sn17'):
            if os.path.isdir(f'../notebooks/data/synthnet/sn17/{synset}'):
                for entity in os.listdir(f'../notebooks/data/synthnet/sn17/{synset}'):
                    if entity == im_entity:
                        im_label = synset

        im_name_new = f'synth_{im_entity}_{im_entity_i:04d}.png'
        dest_path = f'{out_dir}/{im_label}/{im_name_new}'.lower()

        # print(im_entity)
        os.makedirs(f'{out_dir}/{im_label}'.lower(), exist_ok=True)
        shutil.copy(im_path, dest_path)

prepare_synthnet_synth_imgs(IN_DIR, OUT_DIR)
# for path, dns, fns in os.walk("./data/sn17ilsvrc/train/shapenetcore_17/images"):
#     for fn in fns:
#         obj = fn.split('_')[1]
#         os.makedirs(f'{path}/{obj}'.lower(), exist_ok=True)
#         new_fn = f'train{fn[5:]}'
#         shutil.move(f'{path}/{fn}', f"{path}/{obj}/{new_fn}")

# ILSVRC2012_val

In [ ]:
import os
import shutil
from PIL import Image
import untangle

classes = [
    'n02747177', 'n02808440', 'n02843684', 'n02992529', 'n03085013', 'n03207941', 'n03337140', 'n03642806', 'n03691459',
    'n03710193', 'n03759954', 'n03761084', 'n03938244', 'n03991062', 'n04074963', 'n04330267', 'n04554684'
]
in_root = "./data/synthnet/ilsvrc_val_17_prep"

for synset in os.listdir(f'{in_root}/images'):
    for im_name in os.listdir(f'{in_root}/images/{synset}'):
        print(im_name)
        print(f'{in_root}/annotations/{synset}/{im_name[:-5]}.xml')
        annotation = untangle.parse(f'{in_root}/annotations/{synset}/{im_name[:-5]}.xml')
        n = 1
        for ob in annotation.annotation.object:
            ann_synset = ob.name.cdata
            w, h = annotation.annotation.size.width.cdata, annotation.annotation.size.height.cdata
            left, right, top, bottom = int(ob.bndbox.xmin.cdata), int(ob.bndbox.xmax.cdata), int(
                ob.bndbox.ymin.cdata), int(ob.bndbox.ymax.cdata)
            print(ann_synset)
            if ann_synset in classes:
                bb_w = right - left
                bb_h = bottom - top

                im = Image.open(f'{in_root}/images/{synset}/{im_name}')
                # get rectangular bounding box image dimension in px, expand narrow axis to wider axis size
                if bb_w >= bb_h:
                    d = (bb_w - bb_h) / 2
                    left = left
                    top = top - d
                    right = left + bb_w
                    bottom = top + bb_h + d * 2
                if bb_w < bb_h:
                    d = (bb_h - bb_w) / 2
                    left = left - d
                    top = top
                    right = left + bb_w + d * 2
                    bottom = top + bb_h
                im = im.crop((int(left), int(top), int(right), int(bottom)))
                im = im.resize((256, 256))

                ilsvrc_id = im_name[:-5].split("_")[-1]
                out_name = f'{ilsvrc_id}_{n}.png'
                out_dir = f"./data/synthnet/ilsvrc_val_17_resized/{synset}"
                os.makedirs(out_dir, exist_ok=True)
                im.save(f'{out_dir}/{out_name}')

                n = n + 1


# MI3DOR

## MI3DOR - TRAIN SET - SYNTH

In [ ]:
import os
import shutil

ROOT="/home/dennis/Desktop/work/MI3DOR/view/train"
OUT="./data/mi3dor/train_synth/images"
os.makedirs(OUT, exist_ok=True)

for classname in os.listdir(ROOT):
    for entityname in os.listdir(f'{ROOT}/{classname}'):
        os.makedirs(f'{OUT}/{classname}/{entityname}', exist_ok=True)
        for path, dirnames, filenames in os.walk(f'{ROOT}/{classname}/{entityname}'):
            for fname in filenames:
                shutil.copy(f'{path}/{fname}', f'{OUT}/{classname}/{entityname}/{fname}')
    


## MI3DOR - TEST SET - REAL

In [ ]:
import os
import shutil
from PIL import Image

ROOT="/home/dennis/Desktop/work/MI3DOR/image/test"
OUT="./data/mi3dor/test_real/images"
os.makedirs(OUT, exist_ok=True)

TARGET_SIZE = (224, 224)

for classname in os.listdir(ROOT):
    os.makedirs(f'{OUT}/{classname}', exist_ok=True)
    for path, dirnames, filenames in os.walk(f'{ROOT}/{classname}'):
        for fname in filenames:
            im = Image.open(f'{path}/{fname}')
            im = im.resize(TARGET_SIZE)
            im.save(f'{OUT}/{classname}/{fname}')
    


# ModelNet

## ModelNet10

In [ ]:
import os
import shutil

IN_DIR = "/mnt/c/Users/denni/Desktop/work/modelnet10_st_st_none_st_256/render"
OUT = "./data/modelnet10"
for path, dirnames, filenames in os.walk(f'{IN_DIR}'):
    for fname in filenames:
        fname_split = fname.split("_")
        split = fname_split[0]
        label = fname_split[1]
        mesh = f'{fname_split[1]}_{fname_split[2]}'
        # We just hack this in as 'night_stand' is the only two-word-class using _
        if label == 'night':
            label = f'{fname_split[1]}_{fname_split[2]}'
            mesh = f'{fname_split[1]}_{fname_split[2]}_{fname_split[3]}'
        n_img = fname_split[-1].split('.')[0]
        ext = fname_split[-1].split('.')[1]
        if split == 'train':
            out_dir = f'{OUT}/{split}/images/{label}/{mesh}'
        if split == 'test':
            out_dir = f'{OUT}/{split}/images/{label}'
        out_path = f'{out_dir}/{fname}'
        os.makedirs(out_dir, exist_ok=True)
        shutil.copy(f'{path}/{fname}', out_path)


## VisDA 2017

### From Huggingface Imagefolder format -> retrieval pipeline format (with mesh ids) 

In [9]:
import os
import shutil

IN_DIR = "/home/dennis/projects/feature-extractor-finetuning/data/visda2017"
OUT = "./data/datasets/visda2017_meshes"
for path, dirnames, filenames in os.walk(f'{IN_DIR}'):
    for fname in filenames:
        psplit = path.split('/')
        label = psplit[-1]
        split = psplit[-2]
        if split == 'train':
            fsplit = fname.split('__')
            mesh = fsplit[0]
            
            out_dir = f"{OUT}/{split}/images/{label}/{mesh}/"
            os.makedirs(out_dir, exist_ok=True)
            shutil.copy(f"{path}/{fname}", f"{out_dir}/{fname}")

FileNotFoundError: [Errno 2] No such file or directory: './data/datasets/visda2017_meshes/train/images/skateboard/src_1_04225987_5c55e6b6708f730d758f6def7204bd6b//src_1_04225987_5c55e6b6708f730d758f6def7204bd6b/src_1_04225987_5c55e6b6708f730d758f6def7204bd6b__146_236_150_train.png'

### val/test

add labels to dir structure from image_list.txt file

In [14]:
# import os
# import shutil
# import pandas as pd

# DATASET_NAME = 'visda2017'
# DS_DIR = f'data/datasets/{DATASET_NAME}/test/images'

# OUT_ROOT = f'data/{DATASET_NAME}'

# id2label = {id: label for id, label in enumerate(sorted(os.listdir(f"data/datasets/{DATASET_NAME}/val/images")))}
# label2id = {label: id for id, label in id2label.items()}

# print(id2label)
# print(label2id)

# df = pd.read_csv(f"data/datasets/{DATASET_NAME}/test/assets/image_list.txt", sep=" ", header=None)

# for path, dns, fns in os.walk(DS_DIR):
#     for fn in fns:
#         trunk = path.split("/")[-1]
#         label_id = df.loc[df[0] == f'{trunk}/{fn}'][1].values[0]
#         label = id2label[label_id]
#         out = f'{OUT_ROOT}/test/images/{label}/{fn[:-4]}_{label}_test.jpg'
#         os.makedirs(f'{OUT_ROOT}/test/images/{label}', exist_ok=True)
#         shutil.move(f'{path}/{fn}', out)

{0: 'aeroplane', 1: 'bicycle', 2: 'bus', 3: 'car', 4: 'horse', 5: 'knife', 6: 'motorcycle', 7: 'person', 8: 'plant', 9: 'skateboard', 10: 'train', 11: 'truck'}
{'aeroplane': 0, 'bicycle': 1, 'bus': 2, 'car': 3, 'horse': 4, 'knife': 5, 'motorcycle': 6, 'person': 7, 'plant': 8, 'skateboard': 9, 'train': 10, 'truck': 11}
